## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [9]:
%load_ext autoreload
%autoreload 2

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbol=["AMD", "AAPL"],
    interval="1d",
    start_date="2021-01-01",
    end_date="2022-01-01",
)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
toolbox

Toolbox(symbol=['AMD', 'AAPL'], interval=1d, start_date=2021-01-01, end_date=2022-01-01, provider=yfinance)

In [11]:
result = toolbox.technical.mandelbrot_channel(historical=True, window="1m")



AttributeError: 'DataFrame' object has no attribute 'serialize'

In [32]:
result.to_polars()



date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2024-03-19,"""AAPL""",171.5136,176.080002,177.4196
2024-03-19,"""AMD""",173.188,181.419998,204.3356


## Testing HumblObject Creation

In [23]:
from typing import Any
from pydantic import BaseModel, Field
from pydantic.config import ConfigDict
from pydantic.functional_validators import field_validator

from humbldata.core.standard_models.toolbox.technical.mandelbrot_channel import MandelbrotChannelQueryParams


class YourModel(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    command_params: Any = Field()

    @field_validator("command_params")
    def validate_command_params(cls, v):
        class_name = v.__class__.__name__
        if "QueryParams" in class_name:
            return v
        raise TypeError("Wrong type for 'some_foo', must be subclass of Foo")

    def __repr__(self) -> str:
        """Human readable representation of the object."""
        items = [
            f"{k}: {v}"[:83] + ("..." if len(f"{k}: {v}") > 83 else "")
            for k, v in self.model_dump().items()
        ]
        return f"{self.__class__.__name__}\n\n" + "\n".join(items)

YourModel(command_params=MandelbrotChannelQueryParams())


YourModel

command_params: {'window': '1mo', 'rv_adjustment': True, 'rv_method': 'std', 'rs_me...

In [10]:
from typing import Any
from pydantic import BaseModel, Field
from pydantic.config import ConfigDict
from humbldata.core.standard_models.abstract.query_params import QueryParams

from humbldata.core.standard_models.toolbox.technical.mandelbrot_channel import (
    MandelbrotChannelQueryParams,
)


def extract_subclass_dict(self, attribute_name: str, items: list):
    """
    Extract the dictionary representation of the specified attribute.

    Parameters
    ----------
    attribute_name : str
        The name of the attribute to update in the items list.
    """
    # Check if the attribute exists and has a value
    attribute_value = getattr(self, attribute_name, None)
    if attribute_value:
        # Assuming the attribute has a method called 'model_dump' to get its dictionary representation
        add_item = attribute_value.model_dump()
        add_item_str = str(add_item)
        if len(add_item_str) > 83:
            add_item_str = add_item_str[:83] + "..."
        for i, item in enumerate(items):
            if item.startswith(f"{attribute_name}:"):
                items[i] = f"{attribute_name}: {add_item_str}"
                break

    return items


class YourModel(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    command_params: QueryParams | None = Field(
        default_factory=QueryParams,
    )

    def __repr__(self) -> str:
        """Human readable representation of the object."""
        items = [
            f"{k}: {v}"[:83] + ("..." if len(f"{k}: {v}") > 83 else "")
            for k, v in self.model_dump().items()
        ]

        items = extract_subclass_dict(self, "command_params", items)

        return f"{self.__class__.__name__}\n\n" + "\n".join(items)


params = MandelbrotChannelQueryParams(window="2m")
YourModel(command_params=params)

YourModel

command_params: {'window': '2mo', 'rv_adjustment': True, 'rv_method': 'std', 'rs_method': 'RS', 'rv...

In [ ]:
from pydantic import BaseModel, Field
from pydantic.config import ConfigDict


class QueryParams(BaseModel):
    pass


class subQueryParams(QueryParams):
    test: str = "test"


class YourModel(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    command_params: QueryParams = Field()

    def __repr__(self) -> str:
        """Human readable representation of the object."""
        items = [
            f"{k}: {v}"[:83] + ("..." if len(f"{k}: {v}") > 83 else "")
            for k, v in self.model_dump().items()
        ]

        # Needed to extract subclass items
        if self.command_params:
            add_item = self.command_params.model_dump()
        for i, item in enumerate(items):
            if item.startswith('command_params:'):
                items[i] = f'command_params: {add_item}'
                break  # Assuming only one item with 'command_params:', stop after updating


        return f"{self.__class__.__name__}\n\n" + "\n".join(items)


YourModel(command_params=subQueryParams())

YourModel

command_params: {'test': 'test'}